In [1]:
import tensorflow as tf
import keras
import pandas as pd
import numpy
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import DictVectorizer
import socket, struct
from sklearn.preprocessing import OneHotEncoder, normalize, MinMaxScaler
from sklearn.model_selection import KFold
from keras.layers import LSTM, Dense
from keras.models import Sequential
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import chi2
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
import time
import keras_metrics as km
from keras.layers import merge, Input, Dropout, Concatenate,Activation
from keras.models import Model


Using TensorFlow backend.


In [2]:
data = pd.read_csv('./UNSW-NB15_1.csv', header=None)
data1a = numpy.asarray(data)
data1a = numpy.delete(data1a, 47, 1)
data2 = pd.read_csv('./UNSW-NB15_2.csv', header=None)
data2a = numpy.asarray(data2)
data2a = numpy.delete(data2a, 47, 1)
data3 = pd.read_csv('./UNSW-NB15_3.csv', header=None)
data3a = numpy.asarray(data3)
data3a = numpy.delete(data3a, 47, 1)
#data4 = pd.read_csv('./UNSW-NB15_4.csv', header=None)
#data4a = numpy.asarray(data4)
#data4a = numpy.delete(data4a, 47, 1)
print(len(data1a),len(data2a),len(data3a))
#data1 = numpy.append(data1a,data2a)
#data1 = numpy.append(data1, data3a)
#data1 = numpy.append(data1, data4a)
#print(len(data1))
#print(data1[0])
#data1=numpy.reshape(data1,(2540047,48))
#print(len(data1))
data = []
data.append(data1a)
data.append(data2a)
data.append(data3a)
data = numpy.reshape(data,(2100003,48))

C:\Users\Kulik\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Kulik\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


700001 700001 700001


In [3]:

from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
from keras.callbacks import Callback

class SkMetrics(Callback):
    def __init__(self,batchsize):
        self.batchsize = batchsize
    def on_train_begin(self, logs={}):
        self.confusion = []
        self.precision = []
        self.recall = []
        self.f1s = []
        self.kappa = []        

    def on_epoch_end(self, epoch, logs={}):
        score = numpy.asarray(self.model.predict(self.validation_data[0],batch_size = self.batchsize))
        predict = numpy.round(numpy.asarray(self.model.predict(self.validation_data[0],batch_size = self.batchsize)))
        targ = self.validation_data[1]
        targ = targ.flatten()
        predict = predict.flatten()
        self.confusion.append(confusion_matrix(targ, predict, labels=[0,1]))
        self.precision.append(precision_score(targ, predict))
        self.recall.append(recall_score(targ, predict))
        self.f1s.append(f1_score(targ, predict))
        self.kappa.append(cohen_kappa_score(targ, predict))

def getModel(neurons, layers, inputshape, batchsize):
    vec = Input(batch_shape=(batchsize, inputshape[0],inputshape[1]))
    L1 = LSTM(neurons, return_sequences=True, stateful=True)(vec)
    L1_d = Dropout(0.2)(L1)
    input2 = Concatenate()([vec,L1_d])
    L2 = LSTM(neurons, return_sequences=True, stateful=True)(input2)
    L2_d = Dropout(0.2)(L2)
    #input_d = Concatenate()([L1_d,L2_d])
    input3 = Concatenate()([vec, L2_d])
    L3 = LSTM(neurons, return_sequences=True, stateful=True)(input3)
    L3_d = Dropout(0.2)(L3)
    input_d = Concatenate()([L1_d,L2_d,L3_d])
    dense = Dense(1, activation = 'sigmoid')(input_d)
    model = Model(input=vec, output=dense)
    
    model.compile(optimizer="adam", 
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [4]:
def iptoint(ip):
    return struct.unpack("!I",(socket.inet_aton(ip)))[0] 

def parse(data):
    protocols = dict()
    protocolsNum = 1
    services  = dict()
    servicesNum = 1
    states = dict()
    statesNum = 1
    numMalicious = 0
    numNormal = 0
    x = []
    y = []
    for dataRaw in data:
        dataRaw[0] = iptoint(dataRaw[0])
        dataRaw[2] = iptoint(dataRaw[2])
        #Source IP
        if (type(dataRaw[39]) == str):
            if (dataRaw[39] !=  " "):
                dataRaw[39] = int(dataRaw[39],0)
            else:
                dataRaw[39] = 0
        if (type(dataRaw[1]) == str):
            if (dataRaw[1] != "-"):
                dataRaw[1] = int(dataRaw[1],0)
            else:
                dataRaw[1] = 0
        #Dest IP
        if (type(dataRaw[3]) == str):
            if (dataRaw[3] != "-"):
                dataRaw[3] = int(dataRaw[3],0)
            else:
                dataRaw[3] = 0
        if (dataRaw[4] in protocols):
            dataRaw[4] = protocols.get(dataRaw[4])
        else:
            protocols.update({dataRaw[4]:protocolsNum})
            dataRaw[4] = protocolsNum
            protocolsNum += 1

        if (type(dataRaw[4])!= int):
            print (dataRaw[4])
 
        if (dataRaw[5] in states):
            dataRaw[5] = states.get(dataRaw[5])
        else:
            states.update({dataRaw[5]:statesNum})
            dataRaw[5] = statesNum
            statesNum += 1
        if (type(dataRaw[5])!=int):
            print (dataRaw[5])
        if (dataRaw[13] in services):
            dataRaw[13] = services.get(dataRaw[13])
        else:
            services.update({dataRaw[13]:servicesNum})
            dataRaw[13] = servicesNum
            servicesNum += 1
        if (type(dataRaw[13])!=int):
            print (dataRaw[13])
        if (dataRaw[47] == 1):
            numMalicious = numMalicious + 1
            y.append(1)
        else:
            numNormal = numNormal + 1
            y.append(0)
        i = 0
        for data in dataRaw:
            if (type(data) == str):
                print(dataRaw,data,i)
            i = i+1
        x.append(dataRaw)
    print("normal = ",numNormal)
    print("Malicious = ", numMalicious)
    print(protocols)
    print(states)
    print(services)
    return [x, y]


def optimizeParameters(data, labels, scorefunc):
    # Выбираем k лучших признаков
    selecter = SelectKBest(score_func=scorefunc, k=10)
    selecter.fit(data, labels)
    
    # Какие именно выбрали
    params = []
    index = []
    params = selecter.get_support()
    count = 0
    
    for param in params :
        if param == True :
            index.append(count)
        count += 1
    

    index_file = open("index.txt" , "w")
    for i in index : 
        index_file.write(str(i) + " ")
    index_file.close()

    return selecter.transform(data)

def drawGraphic(dataset, labels, normalCount, malwareCount):

    parametersCount = len(dataset[0])

    normalCountList = [0] * parametersCount
    malwareCountList = [0] * parametersCount

    for i in range(0, len(dataset)):
        if(i < normalCount):
            for j in range(0, parametersCount):
                normalCountList[j] += dataset[i][j]
        else:
            for j in range(0, parametersCount):
                malwareCountList[j] += dataset[i][j]

    for i in range(0, parametersCount):
        normalCountList[i] = normalCountList[i]
        malwareCountList[i] = malwareCountList[i] 

    ts = pd.DataFrame({'normal': normalCountList,
                       'malware': malwareCountList})
    ts.plot()
    plt.show()
    return
def ROC(ROC_MODEL , ROC_X, ROC_Y):
    probs = ROC_MODEL.predict(ROC_X)
    ROC_Y = ROC_Y.flatten()
    probs = probs.flatten()
    FPr, TPr, threshold = metrics.roc_curve(ROC_Y, probs)
    roc_auc = metrics.auc(FPr, TPr)
    lw = 2
    
    plt.figure()
    plt.plot(FPr, TPr, lw = lw , label='Logistic Regression (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.savefig('Log_ROC')
    plt.show()
def CrossValidation(lenOfBlock , X1,X2,X3, Y1, Y2, Y3, neurons, layers, inputshape, epochsIn, batchsize):
    cv = KFold(n_splits = lenOfBlock, random_state = None, shuffle = False)
    TN = 0
    FP = 0
    FN = 0
    TP = 0
    F1 = 0
    Educate = 0.0
    Test = 0.0
    print("Neurons: ", neurons)
    print("Layers: ", layers)
    numblock=0
    for train_index, test_index in cv.split(X1):
        clf = getModel(neurons, layers, inputshape, batchsize)
        print("TRAIN:", train_index, "TEST:", test_index)
        print("Block: ", numblock)
        numblock +=1
        # Обучение
        #sample_weights = []
        #for i in train_index:
        #    flag = 0
        #    for yres in Y[i]:
        #        if yres==1:
        #            flag = 1
        #            break
        #    if flag == 1:
        #        sample_weights.append(1)
        #    else:
        #        sample_weights.append(1)
        start_time = time.time()
        skmetrics = SkMetrics(batchsize)
        for i in range (epochsIn):
            #history = clf.fit(X1[train_index], Y1[train_index], callbacks = [skmetrics],epochs = 1, validation_data = (X[test_index],Y[test_index]), sample_weight= numpy.asarray(sample_weights), batch_size = batchsize)
            #history = clf.fit(X2[train_index], Y2[train_index], callbacks = [skmetrics],epochs = 1, validation_data = (X[test_index],Y[test_index]), sample_weight= numpy.asarray(sample_weights), batch_size = batchsize)
            #history = clf.fit(X3[train_index], Y3[train_index], callbacks = [skmetrics],epochs = 1, validation_data = (X[test_index],Y[test_index]), sample_weight= numpy.asarray(sample_weights), batch_size = batchsize)
            history = clf.fit(X1[train_index], Y1[train_index], callbacks = [skmetrics],epochs = 1, validation_data = (X1[test_index],Y1[test_index]), batch_size = batchsize)
            history = clf.fit(X2[train_index], Y2[train_index], callbacks = [skmetrics],epochs = 1, validation_data = (X2[test_index],Y2[test_index]), batch_size = batchsize)
            history = clf.fit(X3[train_index], Y3[train_index], callbacks = [skmetrics],epochs = 1, validation_data = (X3[test_index],Y3[test_index]), batch_size = batchsize)
        for k in skmetrics.confusion:
            print(k)
        education_time = time.time() - start_time
        try:
            # Тестирование
            start_time = time.time()
            proba1 = numpy.round(numpy.asarray(clf.predict(X1[test_index], batch_size = batchsize)))
            proba2 = numpy.round(numpy.asarray(clf.predict(X2[test_index], batch_size = batchsize)))
            proba3 = numpy.round(numpy.asarray(clf.predict(X3[test_index], batch_size = batchsize)))

            test_time = time.time() - start_time
            
            # Добавляем значения
            Educate += education_time
            Test += test_time
            y1 = Y1[test_index].flatten()
            ROC(clf , X1[test_index], Y1[test_index])
            ROC(clf , X2[test_index], Y2[test_index])
            ROC(clf , X3[test_index], Y3[test_index])
            y2 = Y2[test_index].flatten()
            y3 = Y3[test_index].flatten()
            proba1 = proba1.flatten()
            proba2 = proba2.flatten()
            proba3 = proba3.flatten()
            cn1 = confusion_matrix(y1, proba1, labels=[0,1])
            cn2 = confusion_matrix(y2, proba2, labels=[0,1])
            cn3 = confusion_matrix(y3, proba3, labels=[0,1])
            print(cn1)
            print(cn2)
            print(cn3)
            tn1, fp1, fn1, tp1 = cn1.ravel()
            tn2, fp2, fn2, tp2 = cn2.ravel()
            tn3, fp3, fn3, tp3 = cn3.ravel()
        except ZeroDivisionError:
            print("ZeroDivisionError")
        TN += tn1 + tn2+ tn3
        FP += fp1 +fp2 +fp3
        FN += fn1 +fn2 +fn3
        TP += tp1 + tp2 +tp3
        F1 += (f1_score(y1, proba1 , average='binary'))
        F1 += (f1_score(y2, proba2 , average='binary'))
        F1 += (f1_score(y3, proba3 , average='binary'))
        
    summ = TN + FP + FN + TP
    
    print("\n\n\n")
    print("Block len " + str(lenOfBlock))
    print("True positive " + str(TP / summ), TP)
    print("True negative " + str(TN / summ), TN)
    print("False positive " + str(FP / summ), FP)
    print("False negative " + str(FN / summ), FN)
   
    print("Accuracy " + str((TP + TN)/(summ)))
    print("Precision " + str(TP / (TP + FP))) # Precision
    print("Recall " + str(TP / (TP + FN))) # Recall

    print("F1 score " + str(F1 / lenOfBlock)) # F1 score
    print("Educate time " + str(Educate / lenOfBlock)) # Educate time
    print("Test time " + str(Test / lenOfBlock)) # Test time

    print("\n\n\n")    
    


In [5]:

[x, y] = parse(data)
#[x2, newy2] = parse(data2a)
#[x3, newy3] = parse(data3a)
x = numpy.reshape(x,(3, 700001, 48))
y = numpy.reshape(y,(3, 700001))
newy1 = y[0]
newy2 = y[1]
newy3 = y[2]

x1 = x[0]
x2 = x[1]
x3 = x[2]

newx1 = numpy.delete(numpy.asarray(x1), 47, 1)
newx2 = numpy.delete(numpy.asarray(x2), 47, 1)
newx3 = numpy.delete(numpy.asarray(x3), 47, 1)

print(len(x1))
newx1 = newx1.tolist()
newx2 = newx2.tolist()
newx3 = newx3.tolist()
newy1 = newy1.tolist()
newy2 = newy2.tolist()
newy3 = newy3.tolist()
print(x1[0])
print(newy1[0])

normal =  1867614
Malicious =  232389
{'udp': 1, 'arp': 2, 'tcp': 3, 'ospf': 4, 'icmp': 5, 'igmp': 6, 'sctp': 7, 'udt': 8, 'sep': 9, 'sun-nd': 10, 'swipe': 11, 'mobile': 12, 'pim': 13, 'rtp': 14, 'ipnip': 15, 'ip': 16, 'ggp': 17, 'st2': 18, 'egp': 19, 'cbt': 20, 'emcon': 21, 'nvp': 22, 'igp': 23, 'xnet': 24, 'argus': 25, 'bbn-rcc': 26, 'chaos': 27, 'pup': 28, 'hmp': 29, 'mux': 30, 'dcn': 31, 'prm': 32, 'trunk-1': 33, 'xns-idp': 34, 'trunk-2': 35, 'leaf-1': 36, 'leaf-2': 37, 'irtp': 38, 'rdp': 39, 'iso-tp4': 40, 'netblt': 41, 'mfe-nsp': 42, 'merit-inp': 43, '3pc': 44, 'xtp': 45, 'idpr': 46, 'tp++': 47, 'ddp': 48, 'idpr-cmtp': 49, 'ipv6': 50, 'il': 51, 'idrp': 52, 'ipv6-frag': 53, 'sdrp': 54, 'ipv6-route': 55, 'gre': 56, 'rsvp': 57, 'mhrp': 58, 'bna': 59, 'esp': 60, 'i-nlsp': 61, 'narp': 62, 'ipv6-no': 63, 'tlsp': 64, 'skip': 65, 'ipv6-opts': 66, 'any': 67, 'cftp': 68, 'sat-expak': 69, 'kryptolan': 70, 'rvd': 71, 'ippc': 72, 'sat-mon': 73, 'ipcv': 74, 'visa': 75, 'cpnx': 76, 'cphb': 77, 

In [6]:

print(len(newx1))
while (len(newx1)!=700000):
    newx1.pop()
    newy1.pop()
print(len(newx2))
while (len(newx2)!=700000):
    newx2.pop()
    newy2.pop()
print(len(newx3))
while (len(newx3)!=700000):
    newx3.pop()
    newy3.pop()


700001
700001
700001


In [7]:
print (newx1[0])
print (len(newx1[0]))
print (len(newx1))
min_max_scaler = MinMaxScaler()
newx1 = min_max_scaler.fit_transform(newx1)
znewx2 = min_max_scaler.fit_transform(newx2)
newx3 = min_max_scaler.fit_transform(newx3)
print(newx1[0])
xnew1= numpy.reshape(newx1, (len(newx1)//5, 5, 47))
print(len(newy1))
ynew1 = numpy.reshape(newy1,(len(newx1)//5, 5,1))
print(xnew1[0][0])
xnew2= numpy.reshape(newx2, (len(newx2)//5, 5, 47))
print(len(newy2))
ynew2 = numpy.reshape(newy2,(len(newx2)//5, 5,1))
print(xnew2[0][0])
xnew3= numpy.reshape(newx3, (len(newx3)//5, 5, 47))
print(len(newy3))
ynew3 = numpy.reshape(newy3,(len(newx3)//5, 5,1))
print(xnew3[0][0])

for i in range(len(xnew1)):
    for j in range(len(xnew1[i])):
        for k in range(len(xnew1[i][j])):
            if numpy.isnan(xnew1[i][j][k]):
                xnew1[i][j][k] = 1
            if numpy.isnan(xnew2[i][j][k]):
                xnew2[i][j][k] = 1
            if numpy.isnan(xnew3[i][j][k]):
                xnew3[i][j][k] = 1
print(xnew3[0][0])

[1000734720, 1390, 2511044102, 53, 1, 1, 0.0010550000000000002, 132, 164, 31, 29, 0, 0, 1, 500473.9375, 621800.9375, 2, 2, 0, 0, 0, 0, 66, 82, 0, 0, 0.0, 0.0, 1421927414, 1421927414, 0.017, 0.013000000000000001, 0.0, 0.0, 0.0, 0, 0, 0, 0, 0, 3, 7, 1, 3, 1, 1, 1]
47
700000
[2.71179403e-01 2.12100404e-02 6.52407146e-01 9.83321850e-08
 0.00000000e+00 0.00000000e+00 1.20068681e-07 9.65096199e-06
 1.11904015e-05 1.21568627e-01 1.14173228e-01 0.00000000e+00
 0.00000000e+00 0.00000000e+00 9.50026411e-05 4.82907536e-03
 1.96078431e-04 1.82315406e-04 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 4.38829787e-02 5.46666667e-02
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.29984191e-03 0.00000000e+00 2.83286497e-07 2.18541314e-07
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 4.65116279e-02 1.46341463e-01 0.00000000e+00 4.08163265e-02
 0.00000000e+00 0.00000000e+00 0.00000000e+00]
700000
[2

In [8]:
#CrossValidation(10 , xnew , ynew, 100, 2, (5,47), 2, 50)
numpy.isnan(xnew3[0][0][37])
xnew3[0][0][37] = 1
print (xnew3[0][0][37])

1.0


In [9]:
CrossValidation(5 , xnew1, xnew2, xnew3, ynew1, ynew2, ynew3, 300, 2, (5,47), 3, 50)

Neurons:  300
Layers:  2
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


C:\Users\Kulik\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


TRAIN: [ 28000  28001  28002 ... 139997 139998 139999] TEST: [    0     1     2 ... 27997 27998 27999]
Block:  0
Instructions for updating:
Use tf.cast instead.
Train on 112000 samples, validate on 28000 samples
Epoch 1/1
112000/112000 [==============================] - 305s 3ms/step - loss: 0.0130 - acc: 0.9951 - val_loss: 0.0596 - val_acc: 0.9729
Train on 112000 samples, validate on 28000 samples
Epoch 1/1
112000/112000 [==============================] - 297s 3ms/step - loss: 0.0582 - acc: 0.9731 - val_loss: 4.5485e-05 - val_acc: 1.0000


C:\Users\Kulik\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:373: RuntimeWarning: invalid value encountered in true_divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)


Train on 112000 samples, validate on 28000 samples
Epoch 1/1
112000/112000 [==============================] - 301s 3ms/step - loss: 0.0978 - acc: 0.9536 - val_loss: 0.0977 - val_acc: 0.9560
Train on 112000 samples, validate on 28000 samples
Epoch 1/1
112000/112000 [==============================] - 301s 3ms/step - loss: 0.0061 - acc: 0.9968 - val_loss: 0.0718 - val_acc: 0.9714
Train on 112000 samples, validate on 28000 samples
Epoch 1/1
112000/112000 [==============================] - 301s 3ms/step - loss: 0.0349 - acc: 0.9848 - val_loss: 5.0510e-05 - val_acc: 1.0000
Train on 112000 samples, validate on 28000 samples
Epoch 1/1
112000/112000 [==============================] - 300s 3ms/step - loss: 0.0726 - acc: 0.9691 - val_loss: 0.0852 - val_acc: 0.9612
Train on 112000 samples, validate on 28000 samples
Epoch 1/1
112000/112000 [==============================] - 300s 3ms/step - loss: 0.0057 - acc: 0.9972 - val_loss: 0.0561 - val_acc: 0.9751
Train on 112000 samples, validate on 28000 sam